In [22]:
# 하기 코드실행에 필요한 라이브러리들을 가져오는 코드입니다.

import cv2, imutils, re, sys, math
import xlsxwriter, json, os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from matplotlib import rcParams
import operator
import heapq
from pathlib import Path

In [23]:
# 현재의 디렉토리를 확인합니다. 디렉토리를 확인하고 이미지를 가져올 폴더를 선택합니다.

print(os.getcwd())

g:\내 드라이브\code-20220207T014959Z-001\code\code


In [24]:
# 이미지 디렉토리를 설정합니다.
img_dir = '..\data\processed_images\Part2'

In [29]:
# x axis, y axis 를 찾는 함수입니다.
# 이미지의 픽셀중에서 axis일것으로 판단되는 픽셀들을 찾아, 가장 많은 픽셀이 한 직선 위에 있다면 axis 인것으로 특정합니다.
# 
# 변수 설명: 
# filepath: 이미지 파일을 가져올 디렉토리를 설정하는 변수입니다. 
# threshold: 비슷한 색들을 어느정도 수용할 수 있는 RGB 값의 범위를 정합니다. 하얀색의 rgb값은 255,255,255이며 검은색의 rgb 값은 0,0,0 입니다. 인간의 눈으로 보았을 때 255,255,250 도 흰색으로 보이기 때문에 허용될 r, g, b 값의 차이를 선택하는 변수입니다. (기본 10)
# number_of_elements: 검은색~회색 계열의 픽셀을 가장 많이 갖고 있는 직선의 개수를 선택하는 변수 (기본 100)

def axisreader(filepath, threshold = None, number_of_elements = None):

    if filepath is None:
        return None, None
    
    if threshold is None:
        threshold = 10
        
    if number_of_elements is None:
        number_of_elements = 100
    
    # img를 불러옵니다.
    img = cv2.imread(filepath)

    height, width, channels = img.shape
    crop_img = img[3:height-3, 3:width-3]

# 각각 RGB 값의 value 가 +- 1 인 threshold 만들기.

    higher_threshold = np.median(crop_img, axis=2) + threshold
    lower_threshold = np.median(crop_img, axis=2) - threshold

# RGB 중에 한 값이라도 250을 넘어가면 전부 255,255,255로 처리하고, 0보다 작거나 같은 값은 0,0,0으로 처리해라.
    np.place(higher_threshold, higher_threshold > 250, [255,255,255])
    np.place(lower_threshold, lower_threshold <= 0, [0,0,0])

    higher_threshold = np.rint(higher_threshold).astype(int)
    lower_threshold = np.rint(lower_threshold).astype(int)

    h1 = np.array([higher_threshold, higher_threshold, higher_threshold]).astype(int)
    l1 = np.array([lower_threshold,lower_threshold,lower_threshold]).astype(int)

    h1 = h1.reshape(height-6,width-6,3)
    l1 = l1.reshape(height-6,width-6,3)

# RGB 값이 같거나, threshold 안쪽 범위이거나, white 가 아니라면 1로, 그 외에는 0 으로 처리.

    mask = np.uint8(np.where(((crop_img != 255) & (crop_img == crop_img)) | ((crop_img < h1) & (l1 <= crop_img)) , 1, 0))

# reduce sum으로 각각의 row, column 를 더한 후, 조건을 만족하는 1 값이 가장 많은 row, column 찾기

    col_counts = cv2.reduce(mask, 0, cv2.REDUCE_SUM, dtype=cv2.CV_32SC1)
    row_counts = cv2.reduce(mask, 1, cv2.REDUCE_SUM, dtype=cv2.CV_32SC1)

    col_list = col_counts.flatten().tolist()
    row_list = row_counts.flatten().tolist()

    max_col = max(col_list)
    max_row = max(row_list)

#############################################################################################

    col_set = set(col_list)
    col_list_fromset = list(col_set)

# 이미지 column 중에서 yaxis 일 확률이 높은 픽셀이 많은 상위 n 개 찾기 
    col_list_max = heapq.nlargest(number_of_elements, col_list_fromset)


    col_list_maxnum = []
    for i in col_list_max:
        if i < max(col_list_max)-50:
            continue
        col_list_maxnum.append(i)

    col_list_maxnum_norepeat = []
    for v in col_list_maxnum:
        if v not in col_list_maxnum_norepeat:
            col_list_maxnum_norepeat.append(v)


    col_max_index_listnum = []
    for i in col_list:
        for a in col_list_maxnum_norepeat:
            if i == a:
                x = [index for index in range(
                    len(col_list)) if col_list[index] == i]
                for i in x:
                    col_max_index_listnum.append(i)

    col_max_index_listnum

#############################################################################################
    c1 = round(col_max_index_listnum[0]/3)
    yaxis = (c1, 0, c1, height)
#############################################################################################
    row_set = set(row_list)
    row_list_fromset = list(row_set)

# 이미지 row 중에서 xaxis 일 확률이 높은 픽셀이 많은 상위 n 개 찾기 

    row_list_max = heapq.nlargest(number_of_elements, row_list_fromset)

    row_list_maxnum = []
    for i in row_list_max:
        if i < max(row_list_max)-50:
            continue
        row_list_maxnum.append(i)

    row_list_maxnum_norepeat = []
    for v in row_list_maxnum:
        if v not in row_list_maxnum_norepeat:
            row_list_maxnum_norepeat.append(v)


    row_max_index_listnum = []
    for i in row_list:
        for a in row_list_maxnum_norepeat:
            if i == a:
                x = [index for index in range(len(row_list)) if row_list[index] == i]
                for i in x:
                    row_max_index_listnum.append(i)

    row_max_index_listnum
##############################################################################################
    r1 = round(row_max_index_listnum[-1]/3)
    xaxis = (0, r1, width, r1)


# 선그리기

    cv2.line(crop_img, (width-6 ,round(r1)), (0, round(r1)), color=(0,0,255), thickness=1)
    cv2.line(crop_img, (round(c1),height-6), (round(c1),0), color=(0,0,255), thickness=1)
    
    return xaxis, yaxis

In [31]:
# 그래프의 값을 받아올 구역을 정하는 bounding box를 정의하는 함수입니다.

def bounding_box(xaxis, yaxis):

    if xaxis is None:
        return None, None

    if yaxis is None:
        return None, None

    if xaxis[1]*(yaxis[3]-yaxis[0]) > yaxis[3]*xaxis[2]*0.5:
        a = yaxis[0], yaxis[1]
        b = yaxis[0], xaxis[1]
        c = xaxis[2]-6, xaxis[1] 
        d = xaxis[2]-6, yaxis[1]

    if xaxis[1]*yaxis[1] < yaxis[3]*xaxis[2]*0.5:
        a = yaxis[0], -yaxis[1]
        b = yaxis[0], xaxis[1]
        c = xaxis[2]-6, xaxis[1] 
        d = xaxis[2]-6, -yaxis[1]

    
    
    return a, b, c, d

Predicted data 가져오기

In [32]:
# 사용할 라이브러리 실행하는 코드

import io
import cv2
import requests
import pandas as pd
from PIL import Image
from requests_toolbelt.multipart.encoder import MultipartEncoder

In [33]:
# Bar chart 의 data를 수정후 가져오는 함수입니다. filepath 는 이미지가 있는 폴더 위치입니다.

def revised_prediction(filepath):
    # for index, path in enumerate(Path(img_dir).iterdir()):
    #     if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg') or path.name.endswith('.PNG'):
    #         filepath = img_dir + "/" + path.name

    img = cv2.imread(filepath)
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pilImage = Image.fromarray(image)

    # Convert to JPEG Buffer
    buffered = io.BytesIO()
    pilImage.save(buffered, quality=100, format="JPEG")

    # Build multipart form and post request
    m = MultipartEncoder(fields={'file': ("imageToUpload", buffered.getvalue(), "image/jpeg")})

    response = requests.post("https://detect.roboflow.com/bar_graph_label/1?api_key=YkTWEru4OpfhWfNyoIIS", data=m, headers={'Content-Type': m.content_type})
    prediction_value = response.json()

    # label과 text를 extract 하여 저장해둔 excel 파일 위치를 적습니다.
    sheet_num_data = pd.read_excel("../results/FigureData.xlsx", engine='openpyxl')

    sheet_num = len(sheet_num_data)

    # 엑셀 파일에 저장된 각각의 sheet을 for loop을 통해서 가져옵니다.
    for i in range(sheet_num):
        data = pd.read_excel("../results/FigureData.xlsx", sheet_name= i)

        x_text = data.loc[1][1:]
        x_label = data.loc[2]
        y_text = data.loc[3][1:]
        y_label = data.loc[4]
        legends = data.loc[5][1:]


        # 수정할 x, y 값

        x_label_list =[]
        y_label_list =[]


        for i in range(1,len(y_label)-1):
            try:
                y_label_list.append(float(y_label[i]))
            except ValueError:  
                pass
                
        y_value_range = max(y_label_list)-min(y_label_list)

        for i in range(1,len(x_label)-1):
            try:
                x_label_list.append(float(x_label[i]))
            except ValueError:  
                pass
                
        x_value_range = max(x_label_list)-min(x_label_list)
        
        axes = axisreader(filepath)
        xaxis = axes[0]
        yaxis = axes[1]
        # 0,height
        a = bounding_box(xaxis, yaxis)[0]
        # 0,0
        b = bounding_box(xaxis, yaxis)[1]
        # width,0
        c = bounding_box(xaxis, yaxis)[2]
        # width, height
        d = bounding_box(xaxis, yaxis)[3]
        
        x_axis_length = c[0]-b[0]-20
        y_axis_length = b[1]-a[1]-30

        revised_xvalues = []
        revised_yvalues = []
        revised_class = []

        for i in range(0, len(prediction_value['predictions'])):
            revised_xvalues.append((prediction_value['predictions'][i]['x']-b[0])/x_axis_length*x_value_range)
            revised_yvalues.append((b[1]-(prediction_value['predictions'][i]['y']-prediction_value['predictions'][i]['height']*0.5))/y_axis_length*y_value_range)
            revised_class.append(prediction_value['predictions'][i]['class'])

        revised_prediction = pd.DataFrame({'class':revised_class,
        'x':revised_xvalues,
        'y':revised_yvalues
        })

        chart_format = pd.DataFrame({'legend':legends,
        'x_text':x_text,
        'y_text':y_text})

        revised_prediction = revised_prediction.sort_values(by=['x'], ascending=[True])

        revised_prediction = pd.concat([chart_format,revised_prediction])
        return revised_prediction


In [34]:
# 폴더 안의 각 이미지 별로 결과 확인하기

for index, path in enumerate(Path(img_dir).iterdir()):
    if path.name.endswith('.png') or path.name.endswith('.jpg') or path.name.endswith('.jpeg') or path.name.endswith('.PNG'):       
        filepath = img_dir + "/" + path.name
        print(filepath)
        print(revised_prediction(filepath))

..\data\processed_images\Part2/fig 2.PNG
                   legend x_text     y_text  class   x   y
Unnamed: 1  B/A MDCK-MDR1   Time   Losartan    NaN NaN NaN
Unnamed: 2       B/A MDCK    [h]        (25    NaN NaN NaN
Unnamed: 3       A/B MDCK    NaN  transport    NaN NaN NaN
Unnamed: 4  A/B MDCK-MDR1    NaN      [nmol    NaN NaN NaN
Unnamed: 5            NaN    NaN        NaN    NaN NaN NaN
..\data\processed_images\Part2/fig 3.PNG
                   legend x_text     y_text  class   x   y
Unnamed: 1  B/A MDCK-MDR1   Time   Losartan    NaN NaN NaN
Unnamed: 2       B/A MDCK    [h]        (25    NaN NaN NaN
Unnamed: 3       A/B MDCK    NaN  transport    NaN NaN NaN
Unnamed: 4  A/B MDCK-MDR1    NaN      [nmol    NaN NaN NaN
Unnamed: 5            NaN    NaN        NaN    NaN NaN NaN
..\data\processed_images\Part2/fig 4.PNG
                   legend x_text     y_text  class   x   y
Unnamed: 1  B/A MDCK-MDR1   Time   Losartan    NaN NaN NaN
Unnamed: 2       B/A MDCK    [h]        (25    NaN 

Predicted value 확인 

In [39]:
# data extraction 을 위한 data point object detection 이 잘 이뤄졌는지 확인을 하기 위한 함수

# 이미지 위치 넣기 (폴더 아니고 확인할 이미지 한개의 주소)

img = cv2.imread("../data/processed_images/Part2/fig 3-B.PNG")
image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
pilImage = Image.fromarray(image)

# Convert to JPEG Buffer
buffered = io.BytesIO()
pilImage.save(buffered, quality=100, format="JPEG")

# Build multipart form and post request
m = MultipartEncoder(fields={'file': ("imageToUpload", buffered.getvalue(), "image/jpeg")})

response = requests.post("https://detect.roboflow.com/bar_graph_label/1?api_key=YkTWEru4OpfhWfNyoIIS", data=m, headers={'Content-Type': m.content_type})
prediction_value = response.json()

for i in range(0, len(prediction_value['predictions'])):
    cv2.circle(img, (int(prediction_value['predictions'][i]['x']),int(prediction_value['predictions'][i]['y'])), 10, (255,0,0))

cv2.imshow('check', img)
cv2.waitKey(),
cv2.destroyAllWindows()